In [1]:
from tabulate import tabulate
from binance.spot import Spot
client = Spot()

class spot:
    def __init__(self, name, volume, original_price = 0):
        self.name = name
        self.volume = volume
        self.pair_name = name.replace("/","")
        if original_price == 0:
            self.original_price = self.price()
        else:
            self.original_price = original_price
        
    def price(self):
        if self.name == "USDT/USDT":
            return 1
        else:
            return float(client.klines(self.pair_name, "1m", limit=10)[-1][1])
    
    def current_value(self):
        if self.name == "USDT/USDT":
            return float(1 * self.volume)
        else:
            return float(client.klines(self.pair_name, "1m", limit=10)[-1][1]) * self.volume
        
    def origional_value(self):
        return self.original_price * self.volume
    
    def profit(self):
        return self.current_value()-self.origional_value()
        
class portfolio:
    def __init__(self,holding):
        self.holding = holding
        self.names = [c.name for c in holding]
        self.pair_names = [c.pair_name for c in holding]
        self.volumes = [c.volume for c in holding]
        self.original_prices = [round(c.original_price,2) for c in holding]
        self.origional_values = [round(c.origional_value(),2) for c in holding]
        
    def current_values(self):
        return [round(c.current_value(),2) for c in self.holding]
    
    def prices(self):
        return [round(c.price(),2) for c in self.holding]
    
    def profits(self):
        return [round(c.profit(),2) for c in self.holding]
    
    def weights(self):
        return [round(c.current_value()*100 / sum(self.current_values()),2) for c in self.holding]
    
    def performance(self):
        headers = ["Total"] + self.names
        table = [["volumes", " "] + p.volumes
                 ,["original_prices", " "] + self.original_prices
                 ,["origional_values", " "] + self.origional_values
                 ,["prices", " "] + self.prices()
                 ,[" ", " "]*len(self.names)
                 ,["current_value", round(sum(self.current_values()),2)] + self.current_values()
                 ,["profits", round(sum(self.profits()),2)] + self.profits()
                 ,["weights", str(round(sum(p.weights()),2))+"%"] + [str(x)+"%" for x in self.weights()]]
        print(tabulate(table, headers=headers))

        


In [2]:
p_list = [spot("USDT/USDT",1000)]
p_list.append(spot("BTC/USDT",1,35000))
p_list.append(spot("SOL/USDT",100,50))
p_list.append(spot("MATIC/USDT",100))
p_list.append(spot("UNI/USDT",100))
p_list.append(spot("STX/USDT",100))
p_list.append(spot("DYDX/USDT",100))

p = portfolio(p_list)

In [3]:
p.performance()

                  Total     USDT/USDT    BTC/USDT    SOL/USDT    MATIC/USDT    UNI/USDT    STX/USDT    DYDX/USDT
----------------  --------  -----------  ----------  ----------  ------------  ----------  ----------  -----------
volumes                     1000         1           100         100           100         100         100
original_prices             1            35000       50          0.78          6.2         0.63        3.6
origional_values            1000         35000       5000        78.1          619.9       63.43       359.8
prices                      1            37487.21    57.17       0.78          6.2         0.63        3.6

current_value     45325.44  1000.0       37487.21    5717.0      78.1          619.9       63.43       359.8
profits           3204.21   0.0          2487.21     717.0       0.0           0.0         0.0         0.0
weights           100.01%   2.21%        82.7%       12.62%      0.17%         1.37%       0.14%       0.8%
